# Example Notebook for classifier finder

## 0. set log-level of sam_ml library [can be ignored]

(e.g. debugging)

NOTE: has to happen before importing the sam_ml library

In [1]:
import os
os.environ["SAM_ML_LOG_LEVEL"] = "info"

## 1. libraries

In [2]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from sam_ml.models.classifier import LR
from sam_ml.models.automl import CTest

## 2. data

In [3]:
df = load_iris()
y = pd.Series(df.target)
X = pd.DataFrame(df.data, columns=df.feature_names)
x_train, x_test, y_train, y_test = train_test_split(X,y, train_size=0.80, random_state=42)

## 3. model

### 3.1. create tester class object

CTest is an auto-ml class. You can use it to compare different models and find the best one for your data.

**models**: list of *Classifier* subclass objects or *'all'* (for all integrated wrapper class classifier) or *'basic'* (for a smaller selection of basic classifier)

**vectorizer**, **scaler**, **selector**, **sampler**: CTest init creates *Pipeline* objects out of the given models with the data class parameters given
(look into the *iris_pipeline.ipynb* notebook to see the possible parameters)

In [4]:
tester = CTest("all", scaler="minmax")

get all models in the CTest class object

In [5]:
tester.models

{'LogisticRegression (vec=None, scaler=minmax, selector=None, sampler=None)': Pipeline(vectorizer=None, scaler=Scaler(algorithm='minmax', ), selector=None, sampler=None, model=LR(model_name='LogisticRegression'), model_name='LogisticRegression (vec=None, scaler=minmax, selector=None, sampler=None)'),
 'QuadraticDiscriminantAnalysis (vec=None, scaler=minmax, selector=None, sampler=None)': Pipeline(vectorizer=None, scaler=Scaler(algorithm='minmax', ), selector=None, sampler=None, model=QDA(model_name='QuadraticDiscriminantAnalysis'), model_name='QuadraticDiscriminantAnalysis (vec=None, scaler=minmax, selector=None, sampler=None)'),
 'LinearDiscriminantAnalysis (vec=None, scaler=minmax, selector=None, sampler=None)': Pipeline(vectorizer=None, scaler=Scaler(algorithm='minmax', ), selector=None, sampler=None, model=LDA(model_name='LinearDiscriminantAnalysis'), model_name='LinearDiscriminantAnalysis (vec=None, scaler=minmax, selector=None, sampler=None)'),
 'MLP Classifier (vec=None, scaler=

you can add models

In [6]:
tester.add_model(LR(model_name="LogisticRegression (elasticnet penalty)", penalty="elasticnet", solver="saga", l1_ratio=0.5))

In [7]:
tester.models

{'LogisticRegression (vec=None, scaler=minmax, selector=None, sampler=None)': Pipeline(vectorizer=None, scaler=Scaler(algorithm='minmax', ), selector=None, sampler=None, model=LR(model_name='LogisticRegression'), model_name='LogisticRegression (vec=None, scaler=minmax, selector=None, sampler=None)'),
 'QuadraticDiscriminantAnalysis (vec=None, scaler=minmax, selector=None, sampler=None)': Pipeline(vectorizer=None, scaler=Scaler(algorithm='minmax', ), selector=None, sampler=None, model=QDA(model_name='QuadraticDiscriminantAnalysis'), model_name='QuadraticDiscriminantAnalysis (vec=None, scaler=minmax, selector=None, sampler=None)'),
 'LinearDiscriminantAnalysis (vec=None, scaler=minmax, selector=None, sampler=None)': Pipeline(vectorizer=None, scaler=Scaler(algorithm='minmax', ), selector=None, sampler=None, model=LDA(model_name='LinearDiscriminantAnalysis'), model_name='LinearDiscriminantAnalysis (vec=None, scaler=minmax, selector=None, sampler=None)'),
 'MLP Classifier (vec=None, scaler=

you can remove models

In [8]:
tester.remove_model("LogisticRegression (elasticnet penalty) (vec=None, scaler=minmax, selector=None, sampler=None)")

In [9]:
tester.models

{'LogisticRegression (vec=None, scaler=minmax, selector=None, sampler=None)': Pipeline(vectorizer=None, scaler=Scaler(algorithm='minmax', ), selector=None, sampler=None, model=LR(model_name='LogisticRegression'), model_name='LogisticRegression (vec=None, scaler=minmax, selector=None, sampler=None)'),
 'QuadraticDiscriminantAnalysis (vec=None, scaler=minmax, selector=None, sampler=None)': Pipeline(vectorizer=None, scaler=Scaler(algorithm='minmax', ), selector=None, sampler=None, model=QDA(model_name='QuadraticDiscriminantAnalysis'), model_name='QuadraticDiscriminantAnalysis (vec=None, scaler=minmax, selector=None, sampler=None)'),
 'LinearDiscriminantAnalysis (vec=None, scaler=minmax, selector=None, sampler=None)': Pipeline(vectorizer=None, scaler=Scaler(algorithm='minmax', ), selector=None, sampler=None, model=LDA(model_name='LinearDiscriminantAnalysis'), model_name='LinearDiscriminantAnalysis (vec=None, scaler=minmax, selector=None, sampler=None)'),
 'MLP Classifier (vec=None, scaler=

## 3.1. evaluation of the models

CTest has 3 ways implemented to evaluate the models. Depending on the dataset you can choose which one to use

### 3.1.1. one-vs-all cross validation

**Concept:**

The model will be trained on all datapoints except one and then tested on this last one. This will be repeated for all datapoints so that we have our predictions for all datapoints.

**Advantage:** optimal use of information for training

**Disadvantage:** long train time

This concept is very useful for small datasets (datapoints < 150) because the long train time is still not too long and especially with a small amount of information for the model, it is important to use all the information one has for the training.

In [10]:
tester.eval_models_cv(X ,y , avg="macro", small_data_eval=True)
tester.output_scores_as_pd(sort_by="recall", console_out=False)

Crossvalidation:   0%|          | 0/21 [00:00<?, ?it/s]

,accuracy,precision,recall,s_score,l_score,train_score,train_time
"LinearDiscriminantAnalysis (vec=None, scaler=minmax, selector=None, sampler=None)",0.980000,0.980125,0.980000,9.904373e-01,1.000000,0.980000,0:00:00
"QuadraticDiscriminantAnalysis (vec=None, scaler=minmax, selector=None, sampler=None)",0.973333,0.973825,0.973333,9.894085e-01,1.000000,0.980045,0:00:00
"AdaBoostClassifier (RFC based) (vec=None, scaler=minmax, selector=None, sampler=None)",0.953333,0.953448,0.953333,9.861117e-01,1.000000,1.000000,0:00:00
"KNeighborsClassifier (vec=None, scaler=minmax, selector=None, sampler=None)",0.953333,0.953448,0.953333,9.861117e-01,1.000000,0.960045,0:00:00
"BaggingClassifier (RFC based) (vec=None, scaler=minmax, selector=None, sampler=None)",0.953333,0.953448,0.953333,9.861117e-01,1.000000,0.988367,0:00:00
"GaussianNB (vec=None, scaler=minmax, selector=None, sampler=None)",0.953333,0.953448,0.953333,9.861117e-01,1.000000,0.959418,0:00:00
"ExtraTreesClassifier (vec=None, scaler=minmax, selector=None, sampler=None)",0.953333,0.953448,0.953333,9.861117e-01,1.000000,1.000000,0:00:00
"RandomForestClassifier (vec=None, scaler=minmax, selector=None, sampler=None)",0.953333,0.953448,0.953333,9.861117e-01,1.000000,1.000000,0:00:00
"SupportVectorClassifier (vec=None, scaler=minmax, selector=None, sampler=None)",0.953333,0.953448,0.953333,9.861117e-01,1.000000,0.979732,0:00:00
"GradientBoostingMachine (vec=None, scaler=minmax, selector=None, sampler=None)",0.953333,0.953448,0.953333,9.861117e-01,1.000000,1.000000,0:00:00


### 3.1.2. multiple split crossvalidation

does **cv_num** splits and takes the average values for evaluating the model

In [11]:
tester.eval_models_cv(X, y, avg="macro", small_data_eval=False, cv_num=10)
tester.output_scores_as_pd(sort_by="recall", console_out=False)

Crossvalidation:   0%|          | 0/21 [00:00<?, ?it/s]

,accuracy,precision,recall,s_score,l_score,train_time,train_score
"LinearDiscriminantAnalysis (vec=None, scaler=minmax, selector=None, sampler=None)",0.966667,0.850000,0.833333,0.698512,0.700000,0:00:00,0.979259
"QuadraticDiscriminantAnalysis (vec=None, scaler=minmax, selector=None, sampler=None)",0.966667,0.850000,0.833333,0.698512,0.700000,0:00:00,0.982222
"SupportVectorClassifier (vec=None, scaler=minmax, selector=None, sampler=None)",0.953333,0.800000,0.776667,0.599006,0.600000,0:00:00,0.979259
"RandomForestClassifier (vec=None, scaler=minmax, selector=None, sampler=None)",0.946667,0.800000,0.773333,0.599004,0.600000,0:00:00,1.000000
"ExtraTreesClassifier (vec=None, scaler=minmax, selector=None, sampler=None)",0.946667,0.800000,0.773333,0.599005,0.600000,0:00:00,1.000000
"KNeighborsClassifier (vec=None, scaler=minmax, selector=None, sampler=None)",0.946667,0.800000,0.773333,0.599005,0.600000,0:00:00,0.964444
"GradientBoostingMachine (vec=None, scaler=minmax, selector=None, sampler=None)",0.926667,0.800000,0.763333,0.598997,0.600000,0:00:00,1.000000
"DecisionTreeClassifier (vec=None, scaler=minmax, selector=None, sampler=None)",0.953333,0.766667,0.743333,0.499746,0.500000,0:00:00,1.000000
"GaussianNB (vec=None, scaler=minmax, selector=None, sampler=None)",0.946667,0.766667,0.740000,0.499745,0.500000,0:00:00,0.961481
"AdaBoostClassifier (RFC based) (vec=None, scaler=minmax, selector=None, sampler=None)",0.940000,0.750000,0.720000,0.499498,0.500000,0:00:00,1.000000


### 3.1.3. evaluate on given train-test-split

sometimes it only makes sense to split a dataset in one way then cross validation is useless

In [12]:
tester.eval_models(x_train, y_train, x_test, y_test, avg="macro")
tester.output_scores_as_pd(sort_by="recall", console_out=False)

Evaluation:   0%|          | 0/21 [00:00<?, ?it/s]

,accuracy,precision,recall,s_score,l_score,train_time,train_score
"AdaBoostClassifier (RFC based) (vec=None, scaler=minmax, selector=None, sampler=None)",1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,1.000000
"GradientBoostingMachine (vec=None, scaler=minmax, selector=None, sampler=None)",1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,1.000000
"BaggingClassifier (RFC based) (vec=None, scaler=minmax, selector=None, sampler=None)",1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,0.966667
"BaggingClassifier (DTC based) (vec=None, scaler=minmax, selector=None, sampler=None)",1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,0.950000
"GaussianNB (vec=None, scaler=minmax, selector=None, sampler=None)",1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,0.950000
"ExtraTreesClassifier (vec=None, scaler=minmax, selector=None, sampler=None)",1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,1.000000
"KNeighborsClassifier (vec=None, scaler=minmax, selector=None, sampler=None)",1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,0.958333
"AdaBoostClassifier (DTC based) (vec=None, scaler=minmax, selector=None, sampler=None)",1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,0.966667
"XGBClassifier (vec=None, scaler=minmax, selector=None, sampler=None)",1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,1.000000
"SupportVectorClassifier (vec=None, scaler=minmax, selector=None, sampler=None)",1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,0.975000


### 3.2. find best model

**Idea:**

The find_best_model_randomCV method is using randomCVsearch for every modeltype to find its best hyperparameters and afterwards comparing the results of the modeltypes. The randomCVsearch should be just for a few models per modeltype to save time.

**Useful parameters:**

- you can change the used crossvalidation with **small_data_eval** and if *small_data_eval=False*, you can set the number of CVs with **cv_num**

- with the **scoring** parameter you can choose which metric to look at for searching the best model (you can use **avg**, **secondary_scoring**, **strength**, and **pos_label** to more specify it)

- you can change the number of parameter sets to test for each model type with **n_trails**

- you can change with **leave_loadbar** if the loading bar from the  randomCVsearches of each model type shall disappear after they finished

In [13]:
scores = tester.find_best_model_randomCV(x_train, y_train, x_test, y_test, scoring="recall", avg="macro", small_data_eval=False)

randomCVsearch:   0%|          | 0/21 [00:00<?, ?it/s]

randomCVsearch (LogisticRegression (vec=None, scaler=minmax, selector=None, sampler=None)): 100%|██████████| 5/5 [00:00<00:00, 21.16it/s]

2024-01-15 12:23:06,172 - sam_ml.models.main_auto_ml - INFO - LogisticRegression (vec=None, scaler=minmax, selector=None, sampler=None) - score: 0.9506172839506172 (recall) - parameters: {'C': 63.512210106407046, 'penalty': 'l2', 'solver': 'saga'}



randomCVsearch (QuadraticDiscriminantAnalysis (vec=None, scaler=minmax, selector=None, sampler=None)): 100%|██████████| 5/5 [00:00<00:00, 21.35it/s]

2024-01-15 12:23:06,435 - sam_ml.models.main_auto_ml - INFO - QuadraticDiscriminantAnalysis (vec=None, scaler=minmax, selector=None, sampler=None) - score: 0.9598765432098766 (recall) - parameters: {'reg_param': 0.0}


...


randomCVsearch (BaggingClassifier (LR based) (vec=None, scaler=minmax, selector=None, sampler=None)): 100%|██████████| 5/5 [00:03<00:00,  1.29it/s]

2024-01-15 12:24:17,447 - sam_ml.models.main_auto_ml - INFO - BaggingClassifier (LR based) (vec=None, scaler=minmax, selector=None, sampler=None) - score: 0.9175315425315426 (recall) - parameters: {'bootstrap': True, 'bootstrap_features': False, 'max_features': 1.0, 'max_samples': 1.0, 'n_estimators': 10}



randomCVsearch (XGBClassifier (vec=None, scaler=minmax, selector=None, sampler=None)): 100%|██████████| 5/5 [00:00<00:00,  5.78it/s]

2024-01-15 12:24:18,413 - sam_ml.models.main_auto_ml - INFO - XGBClassifier (vec=None, scaler=minmax, selector=None, sampler=None) - score: 0.9459876543209876 (recall) - parameters: {'colsample_bytree': 1.0, 'gamma': 0.0, 'learning_rate': 0.1, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 100, 'reg_alpha': 0, 'reg_lambda': 1.0}


2024-01-15 12:24:18,587 - sam_ml.models.main_auto_ml - INFO - best model type LogisticRegression (vec=None, scaler=minmax, selector=None, sampler=None) - recall: 1.0 - parameters: {'C': 63.512210106407046, 'penalty': 'l2', 'solver': 'saga'}


In [14]:
tester.output_scores_as_pd(sort_by=["recall", "train_time"], console_out=False)

,accuracy,precision,recall,s_score,l_score,train_time,train_score,best_score (rCVs),best_hyperparameters (rCVs)
"LogisticRegression (vec=None, scaler=minmax, selector=None, sampler=None)",1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,0.967063,0.950617,"{'C': 63.512210106407046, 'penalty': 'l2', 'so..."
"LinearDiscriminantAnalysis (vec=None, scaler=minmax, selector=None, sampler=None)",1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,0.975193,0.981481,{'solver': 'svd'}
"DecisionTreeClassifier (vec=None, scaler=minmax, selector=None, sampler=None)",1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,0.949135,0.939374,"{'criterion': 'gini', 'max_depth': 2, 'max_fea..."
"RandomForestClassifier (vec=None, scaler=minmax, selector=None, sampler=None)",1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,0.966229,0.946318,"{'bootstrap': False, 'criterion': 'gini', 'max..."
"GradientBoostingMachine (vec=None, scaler=minmax, selector=None, sampler=None)",1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,0.983740,0.952932,"{'criterion': 'friedman_mse', 'learning_rate':..."
"AdaBoostClassifier (RFC based) (vec=None, scaler=minmax, selector=None, sampler=None)",1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,1.000000,0.952932,"{'algorithm': 'SAMME', 'estimator__max_depth':..."
"KNeighborsClassifier (vec=None, scaler=minmax, selector=None, sampler=None)",1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,1.000000,0.954255,"{'leaf_size': 19, 'n_neighbors': 5, 'p': 4, 'w..."
"ExtraTreesClassifier (vec=None, scaler=minmax, selector=None, sampler=None)",1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,0.983740,0.952932,"{'bootstrap': False, 'criterion': 'gini', 'max..."
"GaussianNB (vec=None, scaler=minmax, selector=None, sampler=None)",1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,0.949969,0.938051,{'var_smoothing': 1e-09}
"GaussianProcessClassifier (vec=None, scaler=minmax, selector=None, sampler=None)",1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,0.941422,0.932150,"{'max_iter_predict': 47, 'multi_class': 'one_v..."


**Note:** If you are using the sam_ml version with SMAC, you can also use the `find_best_model_smac` method which often has a bit better results

**Note:** If you need to test a lot of different pipelines, for example at the beginning to find the best preprocessing steps, you can use the `find_best_model_mass_search` method which is faster and can test a lot of different models/pipelines. Be aware that this method has lower quality results and is recommended to use for an overview.